Author(s): Piyush Amitabh

Details: this code generates mip and stitches them to visualize the tracks

Created: May 02, 2023

License: GNU GPL v3.0

Comment: it is scope agnostic (KLA/WIL LSM) and os agnostic

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# from scipy import ndimage as ndi
# import pickle
import skimage
from PIL import Image
import tifffile as tiff
from natsort import natsorted

In [ ]:
# from tqdm import tqdm
# import plotly.express as px
# import plotly.graph_objects as go
# import dask

In [ ]:
#ask the user for the inputs

print('Warning: This code ONLY works with single channel z-stack tiff images. It will give unpredictable results with multiple channels.')
print('Warning: This code assumes the image name has channel name and position/region name for stiching')
main_dir = input('Enter the Parent directory where ALL the image stacks are stored: ')

In [ ]:
global findscope_flag
findscope_flag = 0

def find_lsm_scope(img_h, img_w):
    '''Finds LSM Scope and downscaling factor automatically using image height and width.
    Returns: 
    ds_factor_h = downscaling factor in height, 
    ds_factor_w = downscaling factor in width'''

    if img_w==img_h: # probably KLA LSM
        findscope_flag = 1

        ds_factor_h = 2048//img_h
        ds_factor_w = 2048//img_w
        r = 2048%img_h

        if r>0: #implying downscaling factor is in fraction
            findscope_flag = 0
            print("Downscaling factor in fraction. Can't process automatically.")

    elif img_w>img_h: # probably WIL LSM
        findscope_flag = 2

        ds_factor_h = 2160//img_h
        ds_factor_w = 2560//img_w

        if ds_factor_h!=ds_factor_w:
            findscope_flag=0

        r_h = 2160%img_h
        r_w = 2560%img_w
        
        if r_h>0 or r_w>0 : #implying downscaling factor is in fraction
            findscope_flag = 0
            print("Downscaling factor in fraction. Can't process automatically.")

    if findscope_flag==1:
        print('LSM Scope used: KLA')
        print(f'Downscaling factor = {ds_factor_w}')
    elif findscope_flag==2:
        print('LSM Scope used: WIL')
        print(f'Downscaling factor = {ds_factor_w}')

    user_check = input('Is the above information correct?(y/n): ')
    if user_check.casefold()=='n':
        findscope_flag = 0
    else:
        print("Fatal Error: Exiting")
        exit()

    if findscope_flag==0: #couldn't find scope, enter manually
        print("ERROR: Can't determine LSM scope automatically.\nEnter manually")
        findscope_flag = int(input('Enter the scope used:\n1 - KLA LSM Scope\n2 - WIL LSM Scope\nInput (1/2): '))
        if findscope_flag==1 or findscope_flag==2:
            ds_factor_h = int(input('Enter the downscaling factor in height: '))
            ds_factor_w = int(input('Enter the downscaling factor in width: '))
        else:
            print("Fatal Error: Exiting")
            exit()

    return(ds_factor_h, ds_factor_w)

In [ ]:
channel_names = ['GFP', 'RFP']

for root, subfolders, filenames in os.walk(main_dir):
    for filename in filenames:
        # print(f'Reading: {filename}')
        
        filepath = os.path.join(root, filename)
        # print(f'Reading: {filepath}')

        filename_list = filename.split('.')
        og_name = filename_list[0] #first of list=name
        ext = filename_list[-1] #last of list=extension

        if ext=="tif" or ext=="tiff": #tiff files
            print(f'Reading Image: {filepath}')
            read_image = tiff.imread(filepath)
            
            if len(read_image.shape)==2: #single images, assume BF
                if findscope_flag==0:
                    ds_h, ds_w = find_lsm_scope(img_h = read_image.shape[0], img_w = read_image.shape[1])

            elif len(read_image.shape)!=2: #image stacks, create MIP
                #generate max intensity zproject from stack_full
                arr_mip = np.max(read_image, axis=0)

            for ch_name in channel_names:
                if ch_name.casefold() in og_name.casefold():
                    dest = os.path.join(root, ch_name.casefold()+'_mip')
                    if not os.path.exists(dest): #check if the dest exists
                        print("Write path doesn't exist.")
                        os.makedirs(dest)
                        print(f"Directory '{ch_name.casefold()}'_mip created")
                # os.chdir(dest)
                img_mip = Image.fromarray(arr_mip)
                img_mip.save(os.path.join(dest, og_name+'_mip.tif'))
                

In [ ]:
# %% [markdown]
# The pixel spacing in our LSM image is 1µm in the z axis, and  0.1625µm in the x and y axes.

# %%
zd, xd, yd = 1, 0.1625, 0.1625 #zeroth dimension is z
orig_spacing = np.array([zd, xd, yd]) #change to the actual pixel spacing from the microscope
new_spacing = np.array([zd, xd*ds_h, yd*ds_w]) #downscale x&y by n